## Detalles a revisar

In [ ]:
#mappings MinMaxScaler, StandardScaler, LabelEncoder

num_pipeline = Pipeline([
    ("std_scaler", StandardScaler())
])

minimax_p = Pipeline([
    ("mmx_scaler", MinMaxScaler(feature_range = (0,1)))
])

logit_pipeline = ColumnTransformer([
    ("cont", num_pipeline, X_cont),
    ("cat", OneHotEncoder(), X_cat),
    ("poly", PolynomialFeatures(interaction_only=True), train_cols)
])

logit_pipeline2 = ColumnTransformer([
    ("cont", minimax_p, X_cont),
    ("cat", OneHotEncoder(), X_cat)
])

pca_pipeline = ColumnTransformer([
    ("cont", num_pipeline, X_cont),
    ("pcas", PCA(n_components = 2), X_cuentas)
])

pca_pipeline2 = ColumnTransformer([
    ("cont", num_pipeline, X_cuentas),
    ("pcas", PCA(n_components = 2), X_cuentas)
])

In [ ]:
y = df.pkbuena

#y = sigmoid(y)

X_train, X_test, y_train, y_test = train_test_split(df[train_cols],y,
                                                    test_size=0.25, random_state=2020)

In [ ]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# set the tolerance to a large value to make the example faster
stoch = SGDClassifier(random_state=200614)
pipe_sgd = Pipeline(steps=[('pca', pca), ('stochasticgd', stoch), ])

# Parameters of pipelines can be set using ‘__’ separated parameter names: max_iter=1000, tol=1e-3
param_grid = {
    'pca__n_components': [1, 2],
    'stochasticgd__max_iter': [500,450, 420,400, 350, 330, 300],
    'stochasticgd__tol': [1e-3,5e-4,5e-3,5e-2]
}

In [ ]:
search = GridSearchCV(pipe_sgd, param_grid,cv = 10, verbose=True, n_jobs=-1)
search.fit(X_train_res, y_train_res)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
cross_val_score(search, X_train, y_train, cv=5, scoring="recall")

In [ ]:
y_train_pred = cross_val_predict(search, X_train, y_train, cv=5)

In [ ]:
# Plot the PCA spectrum
pca.fit(X_train)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(np.arange(1, pca.n_components_ + 1),
         pca.explained_variance_ratio_, '+', linewidth=2)
ax0.set_ylabel('PCA explained variance ratio')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
ax0.legend(prop=dict(size=12))

In [ ]:
X_train_prep = pca_pipeline2.fit_transform(X_train)

In [ ]:
X_test_prep = pca_pipeline2.fit_transform(X_test)

In [ ]:
#Este paso es para balancear entre clases, aunque no es tan necesario

sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

In [ ]:
X_train_res.shape, X_train.shape